In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail, adjust_index=True)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1, adjust_index=True)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55], 'close': [3, 4], 'C': [5, 6], 'D': [7, 8]}, index=[0,1])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)